**FTR REACTOR RYIELD**

yield of n-order Paraffing ASF Model


Wn=(1-a)*(a^(n-1))

sum(Wn)+H2O(15)+ CO2(2) >100 then Scaling Factor SF

SF=(100-(H2O+CO2))/sum(Wn)

Wn_new=Wn*SF

now sum(Wn_new+H2O+CO2)=100

# FTR Yield Data Generation from biomass result

In [4]:
import numpy as np
import pandas as pd


In [5]:

def calculate_yield_hydrocarbon(n, a):
    yields = []
    for i in range(1, n+1):  # Ensure it includes n
        yields.append((1 - a) * (a**(i-1)) * 100)
    return yields

def print_yield_table(n_yield, H2O, CO2):
    till_c_18_sum=n_yield.sum()+H2O+CO2-n_yield[n_yield.size-1]
    if(n_yield.sum()+H2O+CO2<100):
      n_yield[n_yield.size-1]=100.00-till_c_18_sum
    yield_table = pd.DataFrame(columns=['HC-Range', 'Yield (%)'])

    for i in range(len(n_yield)):
        new_row = pd.DataFrame([{"HC-Range": f"Carbon-{i+1}", "Yield (%)": n_yield[i]}])
        yield_table = pd.concat([yield_table, new_row], ignore_index=True)

    new_rows = pd.DataFrame([['H2O', H2O], ['CO2', CO2]], columns=['HC-Range', 'Yield (%)'])
    yield_table = pd.concat([yield_table, new_rows], ignore_index=True)

    display(yield_table)


# calculation start form here
def calculate_for_biomass(name,H2,CO):
  # Given parameters
  a = 3.00-H2/CO  # Chain growth probability alpha
  H2O = 15  # Water yield
  CO2 = 2    # CO2 yield
  n=19

# Compute ASF-based hydrocarbon yields for C1-C20
  n_yield_less_20 = np.array(calculate_yield_hydrocarbon(n, a))

# Normalize if total exceeds 100%
  all_sum = n_yield_less_20.sum() + H2O + CO2
  if all_sum > 100:
      SF = (100 - (H2O + CO2)) / n_yield_less_20.sum()  # Scaling Factor
      n_yield_less_20 *= SF  # Scale hydrocarbons
  print(f"name: {name}, H2:{H2},CO:{CO}, a={a}")
# Print the final yield table

  print_yield_table(n_yield_less_20, H2O, CO2)

# Verify final sum is exactly 100%
  print(f"Final Total Yield: {n_yield_less_20.sum() + H2O + CO2:.2f}%")


In [ ]:

biomass_result=pd.read_csv("https://docs.google.com/spreadsheets/d/1xogZ0K7gRvvukae3BMCq91AbCyHS6QY6VOzNGRAYKgg/gviz/tq?tqx=out:csv&sheet=biomass-syngas-variation"
)
# biomass_list=np.array(biomass_result["biomass"])

for i in range(len(biomass_result)):
  calculate_for_biomass(biomass_result['biomass'].iloc[i],biomass_result['H2(kmol/hr)'].iloc[i],biomass_result['CO(kmol/hr)'].iloc[i])


# Biomasa vs   H2,CO Flow Rate



In [ ]:
import matplotlib.pyplot as plt

biomass_names=[]
H2=[]
CO=[]
for i in range(len(biomass_result)):
  biomass_names.append(biomass_result['biomass'].iloc[i])
  H2.append(biomass_result['H2(kmol/hr)'].iloc[i])
  CO.append(biomass_result['CO(kmol/hr)'].iloc[i])



br1 = np.arange(len(biomass_names))
br2 = [x + 0.25 for x in br1]
fig=plt.figure(figsize=(12, 5))
plt.bar(br1, H2, color ='r', width = 0.2,
        edgecolor ='grey', label ='H2')
plt.bar(br2, CO, color ='g', width = 0.2,
        edgecolor ='grey', label ='CO')
plt.xlabel('Feed Biomass')
plt.ylabel('kmol/hr')
plt.xticks([r + 0.125 for r in range(len(biomass_names))], biomass_names)
plt.legend()

plt.show()

# Predicting Condition for maximum jet fuel (C8-C16) at fixed feed rate of 20000 kg/hr


We have to create table of


*   biomass
*   maximum product
*   temp at which maximum



In [8]:
maximum_output_table = pd.DataFrame(columns=['Biomass', 'Maxmimum Jet Fuel(Kg/hr)',"Occuring Temperature(deg.C)","Chain Growth Probability"])

def find_maximium_condition(biomass,temp,jet_fuel,a): # a: chain growth probability
  global maximum_output_table # Declare maximum_output_table as global
  max_index=np.array(jet_fuel).argmax()
  new_row = pd.DataFrame([{"Biomass":f"{biomass}", "Maxmimum Jet Fuel(Kg/hr)":f"{jet_fuel[max_index]}","Occuring Temperature(deg.C)":f"{temp[max_index]}","Chain Growth Probability":f"{a}" }])
  maximum_output_table = pd.concat([maximum_output_table, new_row], ignore_index=True)

def show_max_output_table():
  display(maximum_output_table)



# Biomass vs Hydrocarnon range plotting


In [ ]:
sheet_names=["coffee-vs-temp","wood_residue-vs-temp","green-vs-temp","muncipal-vs-temp","pine-vs-temp","food-vs-temp","wood_chip-vs-temp"]

def plot_biomass_vs_temp(index):
  biomass_vs_temp=pd.read_csv(f"https://docs.google.com/spreadsheets/d/1xogZ0K7gRvvukae3BMCq91AbCyHS6QY6VOzNGRAYKgg/gviz/tq?tqx=out:csv&sheet={sheet_names[index]}")
  temp=biomass_vs_temp['Temp(deg.C)']
  a=3-biomass_result['H2(kmol/hr)'].iloc[index]/biomass_result['CO(kmol/hr)'].iloc[index] #chain growth probability
  find_maximium_condition(biomass_names[index],temp,biomass_vs_temp['C8-C16 (kg/hr)'],a) # analysing data to find maximum output obtained
  current_fig=plt.figure(figsize=(8,5))
  plt.plot(temp,biomass_vs_temp['C1-C4 (kg/hr)'],label='C1-C4 (kg/hr)')
  plt.plot(temp,biomass_vs_temp['C5-C7 (kg/hr)'],label='C5-C7 (kg/hr)')
  plt.plot(temp,biomass_vs_temp['C8-C16 (kg/hr)'],label='C8-C16 (kg/hr)')
  plt.plot(temp,biomass_vs_temp['C17+ (kg/hr)'],label='C17+ (kg/hr)')
  plt.xlabel('Temperature(deg.C)')
  plt.ylabel('Hydrocarbon(kg/hr)')
  plt.title(f"{biomass_names[index]} ")
  plt.legend()
  plt.show()

for index in range(len(sheet_names)):
  plot_biomass_vs_temp(index)


show_max_output_table() # printing table where maximum of the output is obtained for each biomass

